# PROBLEM 3 : Gaussian Mixture on toy data

## Read data

In [1]:
import requests


url_A = 'https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/2gaussian.txt'
url_B = 'https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/3gaussian.txt'

response_A = requests.get(url_A)
response_B = requests.get(url_B)


In [204]:
data_A = response_A.text
data_B = response_B.text

with open('2gaussian.txt', 'w') as file:
    file.write(data_A)

with open('3gaussian.txt', 'w') as file:
    file.write(data_B)

In [205]:
import pandas as pd

d_A = {'col1':[], 'col2':[]}

with open("2gaussian.txt", "r") as file:
    for line in file:
        values = line.split(' ')
        d_A['col1'].append(float(values[0]))
        d_A['col2'].append(float(values[1]))

d_B = {'col1':[], 'col2':[]}
with open("3gaussian.txt", "r") as file:
    for line in file:
        values = line.split(' ')
        d_B['col1'].append(float(values[0]))
        d_B['col2'].append(float(values[1]))

df_A = pd.DataFrame(d_A)
df_B = pd.DataFrame(d_B)


In [206]:
df_A.isnull().sum()

col1    0
col2    0
dtype: int64

In [207]:
df_B.isnull().sum()


col1    0
col2    0
dtype: int64

In [208]:
df_A.to_csv('2gaussian.csv', index=False)
df_B.to_csv('3gaussian.csv', index=False)


## A

In [141]:
import pandas as pd
df = pd.read_csv('2gaussian.csv')

In [147]:
import numpy as np
import random
from scipy.stats import multivariate_normal

def initialize_parameters(arr, K):
    indices = np.random.choice(len(arr), size=K, replace=False)
    mus = arr[indices]
    covs = [np.eye(arr.shape[1]) for _ in range(K)]
    wi = np.ones(K) / K
    return mus, np.array(covs), wi

def e_step(mu, cov, arr, wi, K):
    probs = []
    n = arr.shape[0]
    for i in range(K):
        probs.append(multivariate_normal.pdf(arr, mean=mu[i], cov=cov[i]))
    probs = np.array(probs)
    probs = probs * wi.reshape(K, 1)
    probs_sum = np.sum(probs, axis=0)
    resp = probs / (probs_sum + 1e-10)
    return resp

def m_step(arr, pi_vals, K):
    covs = []
    mus = []
    n = arr.shape[0]
    d = arr.shape[1]

    for k in range(K):
        kth_resp = pi_vals[k]
        kth_resp_sum = np.sum(kth_resp)
        mu = np.sum(arr * kth_resp.reshape(n,1), axis=0) / kth_resp_sum
        x = arr - mu
        cov = (x.T @ (x * kth_resp.reshape(n,1))) / kth_resp_sum + np.eye(d)*1e-6
        mus.append(mu)
        covs.append(cov)
        wi = np.sum(pi_vals, axis=1) / n
    return mus, covs, wi

def log_likelihood(arr, mus, covs, wi, K):
    obj_val = 0
    n = arr.shape[0]
    for i in range(K):
        probs = multivariate_normal.pdf(arr, mean=mus[i], cov=covs[i])
        obj_val += np.log(np.sum(wi[i] * probs))
    return obj_val

def converge(arr, K, max_itr=100, tol=1e-6):
    mus, covs, wi = initialize_parameters(arr, K)
    prev_log_l = -np.inf
    for itr in range(max_itr):
        resp = e_step(mus, covs, arr, wi, K)
        mus, covs, wi = m_step(arr, resp, K)
        log_l = log_likelihood(arr, mus, covs, wi, K)
        print("Iteration {itr+1}: Log-likelihood = {log_l}")
        if np.abs(log_l - prev_log_l) < tol:
            break
        prev_log_l = log_l
    return mus, covs, wi

In [148]:
K = 2
arr = df.to_numpy()
mus, covs, wi = converge(arr, K, max_itr=40)

Iteration 1: Log-likelihood = 9.012502414013502
Iteration 2: Log-likelihood = 9.02874102010129
Iteration 3: Log-likelihood = 9.021559291886616
Iteration 4: Log-likelihood = 9.01470143041412
Iteration 5: Log-likelihood = 9.009561367986356
Iteration 6: Log-likelihood = 9.005854218822412
Iteration 7: Log-likelihood = 9.00322987563401
Iteration 8: Log-likelihood = 9.00139695714168
Iteration 9: Log-likelihood = 9.000129567698771
Iteration 10: Log-likelihood = 8.999259555332912
Iteration 11: Log-likelihood = 8.998665380720004
Iteration 12: Log-likelihood = 8.998261033351964
Iteration 13: Log-likelihood = 8.997986541272734
Iteration 14: Log-likelihood = 8.997800513952615
Iteration 15: Log-likelihood = 8.997674584338077
Iteration 16: Log-likelihood = 8.997589403393153
Iteration 17: Log-likelihood = 8.99753181579645
Iteration 18: Log-likelihood = 8.997492896849975
Iteration 19: Log-likelihood = 8.997466600905074
Iteration 20: Log-likelihood = 8.997448836694975
Iteration 21: Log-likelihood = 8.9

In [149]:
mus

[array([7.01315024, 3.98313534]), array([2.99413566, 3.0520961 ])]

Results are below

In [150]:
covs

[array([[0.97475662, 0.49746833],
        [0.49746833, 1.00114191]]),
 array([[1.01024165, 0.02719055],
        [0.02719055, 2.93782075]])]

In [151]:
wi

array([0.66520359, 0.3347964 ])

# B

In [119]:
import pandas as pd
df = pd.read_csv('3gaussian.csv')

In [120]:
K = 3
arr = df.to_numpy()
mus, covs, wi = converge(arr, K, max_itr=50)

Iteration 1: Log-likelihood = 10.107867650644387
Iteration 2: Log-likelihood = 10.25554086781822
Iteration 3: Log-likelihood = 10.377663639756275
Iteration 4: Log-likelihood = 10.48024410708884
Iteration 5: Log-likelihood = 10.566858010553448
Iteration 6: Log-likelihood = 10.642344397133426
Iteration 7: Log-likelihood = 10.711913999175497
Iteration 8: Log-likelihood = 10.780751845097168
Iteration 9: Log-likelihood = 10.854283774172789
Iteration 10: Log-likelihood = 10.937945471209266
Iteration 11: Log-likelihood = 11.036011886804465
Iteration 12: Log-likelihood = 11.150843624250928
Iteration 13: Log-likelihood = 11.28300364793494
Iteration 14: Log-likelihood = 11.431898709936812
Iteration 15: Log-likelihood = 11.595799674240356
Iteration 16: Log-likelihood = 11.772675921448567
Iteration 17: Log-likelihood = 11.96035631801788
Iteration 18: Log-likelihood = 12.15213362754347
Iteration 19: Log-likelihood = 12.334118585976675
Iteration 20: Log-likelihood = 12.491265517708886
Iteration 21: 

In [121]:
mus

[array([5.01091469, 7.00074739]),
 array([3.03804719, 3.04382128]),
 array([7.02120915, 4.01527054])]

Result are below

In [122]:
covs

[array([[0.98070786, 0.18578253],
        [0.18578253, 0.97551657]]),
 array([[1.02723265, 0.02376725],
        [0.02376725, 3.3753003 ]]),
 array([[0.99101181, 0.50118987],
        [0.50118987, 0.99567759]])]

In [123]:
wi

array([0.49630332, 0.20521438, 0.29848228])